# Β2.iii) Εξαγωγή Τίτλων Clusters με χρήση LLM

Αυτό το notebook εκτελεί την εξαγωγή περιγραφικών τίτλων για τα clusters που δημιουργήθηκαν στην ενότητα Β2.ii, χρησιμοποιώντας Large Language Models (LLM).

**Κύρια Χαρακτηριστικά:**
- **LLM Model**: Google Gemma-3-4b-it:free μέσω OpenRouter API
- **Input**: Clusters από το αρχείο `documents_with_clusters.csv`
- **Μέθοδοι**: Δύο προσεγγίσεις για επιλογή εκπροσωπευτικών εγγράφων
- **Output**: Τίτλοι για κάθε cluster σε CSV format

**Διαδικασία:**
1. Φόρτωση δεδομένων με cluster assignments
2. Re-fitting του TF-IDF vectorizer και K-means μοντέλου
3. Επιλογή εκπροσωπευτικών εγγράφων (κεντροειδής vs τυχαία)
4. Εξαγωγή τίτλων μέσω LLM calls
5. Αποθήκευση και ανάλυση αποτελεσμάτων

In [14]:
import csv
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
import re
import random
import sys
import time
import warnings

warnings.filterwarnings('ignore')

# Εισαγωγή llm_utils από το parent directory
SCRIPT_DIR = os.path.dirname(os.path.abspath(''))
LLM_UTILS_DIR = os.path.join(SCRIPT_DIR, 'merosB2')
sys.path.append(LLM_UTILS_DIR)

try:
    from llm_utils import send_message_to_llm
    print(f"Successfully imported send_message_to_llm from: {os.path.join(LLM_UTILS_DIR, 'llm_utils.py')}")
    LLM_AVAILABLE = True
except ImportError as e:
    print(f"Error importing send_message_to_llm: {e}")
    print("Will use mock responses for demonstration")
    send_message_to_llm = None
    LLM_AVAILABLE = False

print("Libraries imported successfully")

Successfully imported send_message_to_llm from: c:\Users\USER\Desktop\PROJECTS\ML_GreekLegalDocs\merosB2\llm_utils.py
Libraries imported successfully


## 1. Παραμετροποίηση

Ορισμός των βασικών παραμέτρων για την εξαγωγή τίτλων clusters με LLM.

In [15]:
# --- Configuration ---
CHOSEN_K = 21
TEXT_COLUMN_TO_USE = 'summary'
RANDOM_STATE = 42

# Fix the path to point to the correct location of documents_with_clusters.csv
# The file is in the same directory as this notebook (merosB2 folder)
INPUT_CSV_FILE = os.path.join(os.path.dirname(os.path.abspath('')), 'merosB2', 'documents_with_clusters.csv')

# Alternative: Use relative path from notebook location
# INPUT_CSV_FILE = 'documents_with_clusters.csv'

# Λίστες με συγκεκριμένα cluster_ids για επεξεργασία
# Αν είναι άδειες, θα επεξεργαστούν όλα τα clusters
CLUSTERS_TO_PROCESS_CENTROID = []  # Μέθοδος κεντροειδούς
CLUSTERS_TO_PROCESS_RANDOM = []    # Τυχαία μέθοδος

# TF-IDF Parameters
TFIDF_MAX_DF = 0.90
TFIDF_MIN_DF = 5
TFIDF_NGRAM_RANGE = (1, 2)

# LLM Prompt Template για πολλαπλά έγγραφα
LLM_MULTI_DOC_PROMPT_TEMPLATE = (
    "Σου δίνονται {NUM_SUMMARIES} περιλήψεις νομικών αποφάσεων που ανήκουν στην ίδια θεματική κατηγορία:\n\n"
    "{SUMMARIES_TEXT}\n\n"
    "Βάσει αυτών των περιλήψεων, ποιο είναι το κεντρικό, ενοποιημένο θέμα που τις καλύπτει όλες; "
    "Απάντησε με έναν σύντομο τίτλο (ιδανικά 3-7 λέξεις) στην μορφή: 'Θέμα:\\n\"<ο τίτλος σου εδώ>\"'. "
    "Μην προσθέτεις εισαγωγικά σχόλια, εξηγήσεις ή περιττές φράσεις."
)

# Markers για αποτελέσματα LLM calls
LLM_CALL_FAILURE_MARKER = "_LLM_CALL_FAILED_"

print("Configuration:")
print(f"- K value: {CHOSEN_K}")
print(f"- Text column: {TEXT_COLUMN_TO_USE}")
print(f"- Input CSV: {INPUT_CSV_FILE}")
print(f"- CSV file exists: {os.path.exists(INPUT_CSV_FILE)}")
print(f"- LLM Available: {LLM_AVAILABLE}")
print(f"- Processing clusters (Centroid): {'All' if not CLUSTERS_TO_PROCESS_CENTROID else CLUSTERS_TO_PROCESS_CENTROID}")
print(f"- Processing clusters (Random): {'All' if not CLUSTERS_TO_PROCESS_RANDOM else CLUSTERS_TO_PROCESS_RANDOM}")

Configuration:
- K value: 21
- Text column: summary
- Input CSV: c:\Users\USER\Desktop\PROJECTS\ML_GreekLegalDocs\merosB2\documents_with_clusters.csv
- CSV file exists: True
- LLM Available: True
- Processing clusters (Centroid): All
- Processing clusters (Random): All


## 2. Φόρτωση και Προετοιμασία Δεδομένων

Φορτώνουμε το αρχείο με τα cluster assignments και προετοιμάζουμε τα δεδομένα για την εξαγωγή τίτλων.

In [16]:
def load_data(csv_path):
    """Loads data with cluster assignments."""
    print(f"Loading data from '{csv_path}'...")
    try:
        df = pd.read_csv(csv_path)
        print(f"Data loaded successfully. Shape: {df.shape}")
        if 'cluster_id' not in df.columns:
            raise ValueError("CSV file must contain 'cluster_id' column.")
        if TEXT_COLUMN_TO_USE not in df.columns:
            raise ValueError(f"CSV file must contain '{TEXT_COLUMN_TO_USE}' column.")
        df[TEXT_COLUMN_TO_USE] = df[TEXT_COLUMN_TO_USE].fillna('')
        return df
    except FileNotFoundError:
        print(f"Error: File not found at '{csv_path}'. Please ensure the CSV file exists at this location.")
        return None
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

# Φόρτωση δεδομένων
df_full = load_data(INPUT_CSV_FILE)

if df_full is not None:
    print(f"\nDataset info:")
    print(f"- Total documents: {len(df_full)}")
    print(f"- Unique clusters: {df_full['cluster_id'].nunique()}")
    print(f"- Cluster range: {df_full['cluster_id'].min()} to {df_full['cluster_id'].max()}")
    print(f"- Non-null summaries: {df_full[TEXT_COLUMN_TO_USE].notna().sum()}")
    
    # Cluster distribution
    cluster_counts = df_full['cluster_id'].value_counts().sort_index()
    print(f"\nCluster size distribution:")
    print(f"- Smallest cluster: {cluster_counts.min()} documents")
    print(f"- Largest cluster: {cluster_counts.max()} documents")
    print(f"- Average cluster size: {cluster_counts.mean():.1f} documents")
else:
    print("Failed to load data. Cannot proceed.")

Loading data from 'c:\Users\USER\Desktop\PROJECTS\ML_GreekLegalDocs\merosB2\documents_with_clusters.csv'...
Data loaded successfully. Shape: (8395, 6)

Dataset info:
- Total documents: 8395
- Unique clusters: 21
- Cluster range: 0 to 20
- Non-null summaries: 8395

Cluster size distribution:
- Smallest cluster: 22 documents
- Largest cluster: 1682 documents
- Average cluster size: 399.8 documents
Data loaded successfully. Shape: (8395, 6)

Dataset info:
- Total documents: 8395
- Unique clusters: 21
- Cluster range: 0 to 20
- Non-null summaries: 8395

Cluster size distribution:
- Smallest cluster: 22 documents
- Largest cluster: 1682 documents
- Average cluster size: 399.8 documents


## 3. Re-fitting του TF-IDF και K-means

Επαναεκπαίδευση του TF-IDF vectorizer και του K-means μοντέλου για τον υπολογισμό των κεντροειδών.

In [17]:
def refit_vectorizer_and_kmeans(df_full, current_k_value):
    """Re-fits TF-IDF and K-means."""
    print("\nRe-fitting TF-IDF vectorizer...")
    vectorizer = TfidfVectorizer(
        max_df=TFIDF_MAX_DF, 
        min_df=TFIDF_MIN_DF, 
        ngram_range=TFIDF_NGRAM_RANGE, 
        stop_words=None
    )
    X_full = vectorizer.fit_transform(df_full[TEXT_COLUMN_TO_USE])
    print(f"TF-IDF matrix re-fitted. Shape: {X_full.shape}")
    print(f"Number of features: {len(vectorizer.get_feature_names_out())}")

    print(f"\nRe-fitting K-means model for K={current_k_value}...")
    kmeans_model = KMeans(
        n_clusters=current_k_value, 
        init='k-means++', 
        n_init='auto', 
        random_state=RANDOM_STATE
    )
    kmeans_model.fit(X_full)
    print("K-means model re-fitted.")
    return X_full, vectorizer, kmeans_model

# Καθορισμός K value από τα δεδομένα
if df_full is not None:
    if 'cluster_id' in df_full.columns and df_full['cluster_id'].nunique() > 0:
        k_for_refit = df_full['cluster_id'].nunique()
        print(f"\nK value for refitting K-means (determined from CSV): {k_for_refit}")
    else:
        print(f"\nUsing CHOSEN_K = {CHOSEN_K} as fallback")
        k_for_refit = CHOSEN_K 

    # Re-fitting
    X_full, vectorizer, kmeans_model = refit_vectorizer_and_kmeans(df_full, k_for_refit)
    
    print(f"\nModel refitting completed:")
    print(f"- TF-IDF features: {X_full.shape[1]}")
    print(f"- K-means clusters: {kmeans_model.n_clusters}")
    print(f"- Centroids shape: {kmeans_model.cluster_centers_.shape}")


K value for refitting K-means (determined from CSV): 21

Re-fitting TF-IDF vectorizer...
TF-IDF matrix re-fitted. Shape: (8395, 23149)
Number of features: 23149

Re-fitting K-means model for K=21...
TF-IDF matrix re-fitted. Shape: (8395, 23149)
Number of features: 23149

Re-fitting K-means model for K=21...
K-means model re-fitted.

Model refitting completed:
- TF-IDF features: 23149
- K-means clusters: 21
- Centroids shape: (21, 23149)
K-means model re-fitted.

Model refitting completed:
- TF-IDF features: 23149
- K-means clusters: 21
- Centroids shape: (21, 23149)


## 4. Συναρτήσεις για Επιλογή Εκπροσωπευτικών Εγγράφων

Ορίζουμε συναρτήσεις για την επιλογή εκπροσωπευτικών εγγράφων από κάθε cluster με δύο μεθόδους:
1. **Κεντροειδής**: Επιλογή εγγράφων πιο κοντά στο κεντροειδές του cluster
2. **Τυχαία**: Τυχαία επιλογή εγγράφων από το cluster

In [18]:
def get_docs_closest_to_centroid(df_cluster_docs, X_cluster_docs, centroid_vector, n_docs=3):
    """Finds n_docs closest to the centroid."""
    if X_cluster_docs.shape[0] == 0: 
        return pd.DataFrame()
    
    distances = euclidean_distances(X_cluster_docs, centroid_vector.reshape(1, -1))
    closest_indices_in_cluster = np.argsort(distances.ravel())[:n_docs]
    return df_cluster_docs.iloc[closest_indices_in_cluster]

def get_random_docs_from_cluster(df_cluster_docs, n_docs=3, random_state=None):
    """Selects n_docs randomly."""
    if df_cluster_docs.shape[0] == 0: 
        return pd.DataFrame()
    if df_cluster_docs.shape[0] <= n_docs: 
        return df_cluster_docs
    return df_cluster_docs.sample(n=n_docs, random_state=random_state)

# Test των συναρτήσεων με το πρώτο cluster
if df_full is not None and 'kmeans_model' in locals():
    test_cluster_id = 0
    df_test_cluster = df_full[df_full['cluster_id'] == test_cluster_id].copy()
    
    if len(df_test_cluster) > 0:
        print(f"\nTesting document selection functions with cluster {test_cluster_id}:")
        print(f"- Cluster size: {len(df_test_cluster)} documents")
        
        # Test centroid method
        test_indices = df_test_cluster.index
        X_test_cluster = X_full[test_indices]
        test_centroid = kmeans_model.cluster_centers_[test_cluster_id]
        
        closest_docs = get_docs_closest_to_centroid(df_test_cluster, X_test_cluster, test_centroid, n_docs=3)
        print(f"- Closest to centroid: {len(closest_docs)} documents selected")
        
        # Test random method
        random_docs = get_random_docs_from_cluster(df_test_cluster, n_docs=3, random_state=RANDOM_STATE)
        print(f"- Random selection: {len(random_docs)} documents selected")
        
        print("Document selection functions working correctly.")


Testing document selection functions with cluster 0:
- Cluster size: 264 documents
- Closest to centroid: 3 documents selected
- Random selection: 3 documents selected
Document selection functions working correctly.


## 5. LLM Integration και Response Parsing

Ορίζουμε συναρτήσεις για την επικοινωνία με το LLM και την ανάλυση των responses.

In [19]:
def call_actual_llm(prompt_text: str) -> str:
    """
    Calls the actual LLM using send_message_to_llm from llm_utils, with retries.
    Returns the raw response string from LLM on success, or LLM_CALL_FAILURE_MARKER.
    """
    if send_message_to_llm is None:
        print("    LLM utility not available. Returning mock response.")
        # Δημιουργία mock response για demonstration
        keywords = re.findall(r'\b[Α-Ωα-ωίϊΐόάέύϋΰώΊΪΌΆΈΎΫΏ]{5,}\b', 
                             prompt_text.split("Περίληψη 1:")[-1] if "Περίληψη 1:" in prompt_text else prompt_text)
        theme_content = "Mock: Σχετικά με " + ", ".join(keywords[:2]) if keywords else "Mock: Γενικό θέμα"
        return f"Θέμα:\n\"{theme_content}\""

    effective_system_prompt = (
        "Είσαι ένας βοηθός που εξάγει το κεντρικό θέμα από τα παρεχόμενα κείμενα. "
        "Απάντησε ΜΟΝΟ με το θέμα ακολουθώντας τη μορφή <ο τίτλος>. Για παράδειγμα: "
        "Input: <Κείμενο>, Output: <ο τίτλος> "
        "Μην προσθέτεις εισαγωγικά σχόλια, εξηγήσεις ή περιττές φράσεις."
    )

    MAX_RETRIES = 3
    RETRY_DELAY_SECONDS = 3

    for attempt in range(MAX_RETRIES):
        print(f"    LLM call attempt {attempt + 1}/{MAX_RETRIES}...")
        try:
            raw_response = send_message_to_llm(
                user_message=prompt_text,
                system_message=effective_system_prompt,
            )
            
            if raw_response: 
                if raw_response.strip().startswith("Θέμα:") or raw_response.strip().lower().startswith("θέμα:"): 
                    print(f"    LLM attempt {attempt + 1} successful and response format seems OK.")
                    return raw_response.strip() 
                else:
                    print(f"    LLM attempt {attempt + 1} response malformed: '{raw_response[:100]}...'")
                    if attempt == MAX_RETRIES - 1: 
                        print(f"    LLM returned malformed response after all retries. Passing as is.")
                        return raw_response.strip() 
            
            print(f"    LLM call attempt {attempt + 1} returned empty response.")
            if attempt < MAX_RETRIES - 1:
                print(f"    Retrying in {RETRY_DELAY_SECONDS}s...")
                time.sleep(RETRY_DELAY_SECONDS)
                
        except Exception as e:
            print(f"    LLM call attempt {attempt + 1} failed with error: {e}")
            if attempt < MAX_RETRIES - 1:
                print(f"    Retrying in {RETRY_DELAY_SECONDS}s...")
                time.sleep(RETRY_DELAY_SECONDS)
            
    print(f"    LLM call failed after {MAX_RETRIES} attempts.")
    return LLM_CALL_FAILURE_MARKER

def parse_llm_response(raw_llm_output: str) -> str:
    """
    Parses the raw LLM output. Returns a clean theme string, or an error/status string.
    """
    if raw_llm_output == LLM_CALL_FAILURE_MARKER:
        return "Αποτυχία επικοινωνίας με LLM"

    # Αναζήτηση pattern "Θέμα:" ακολουθούμενο από το θέμα
    match = re.search(r"Θέμα:\s*\n?(.*?)(?:\n\n|$)", raw_llm_output, re.DOTALL | re.IGNORECASE)
    
    theme_text = ""

    if match:
        theme_text = match.group(1).strip() 
    else:
        # Fallback parsing
        lower_raw = raw_llm_output.lower()
        thema_keyword = "θέμα:"
        if thema_keyword in lower_raw:
            last_occurrence_index = lower_raw.rfind(thema_keyword)
            theme_text = raw_llm_output[last_occurrence_index + len(thema_keyword):].strip()
        else: 
            print(f"    Warning: 'Θέμα:' prefix not found. Treating raw response as theme.")
            theme_text = raw_llm_output

    # Καθαρισμός του theme
    cleaned_theme = theme_text.strip()
    
    # Αφαίρεση literal "\\n" characters
    if '\\n' in cleaned_theme: 
        while cleaned_theme.startswith('\\n'):
            cleaned_theme = cleaned_theme[2:].strip()

    # Αφαίρεση εισαγωγικών αν υπάρχουν
    if cleaned_theme.startswith('"') and cleaned_theme.endswith('"') and len(cleaned_theme) > 1:
        cleaned_theme = cleaned_theme[1:-1].strip()
    
    # Τελικός καθαρισμός
    cleaned_theme = cleaned_theme.strip()

    if not cleaned_theme:
        return "Κενό θέμα από LLM"
    
    # Αφαίρεση τυχόν υπολειπόμενων "Θέμα:" prefixes
    if cleaned_theme.startswith("Θέμα:") or cleaned_theme.lower().startswith("θέμα:"):
        cleaned_theme = re.sub(r"^(Θέμα:|θέμα:)\s*\n?", "", cleaned_theme, flags=re.IGNORECASE).strip()
        if not cleaned_theme:
            return "Κενό θέμα από LLM (μετά από διόρθωση)"

    return cleaned_theme

# Test της LLM integration
print("\nTesting LLM integration...")
test_prompt = "Test prompt για δοκιμή LLM integration"
test_response = call_actual_llm(test_prompt)
print(f"LLM test response: {test_response[:100]}...")
parsed_test = parse_llm_response(test_response)
print(f"Parsed test response: {parsed_test}")


Testing LLM integration...
    LLM call attempt 1/3...
Sending request to OpenRouter API using model: google/gemma-3-4b-it:free
API response received successfully
    LLM attempt 1 response malformed: 'Δοκιμαστική προτροπή για δοκιμή ενσωμάτωσης LLM...'
    LLM call attempt 1 returned empty response.
    Retrying in 3s...
API response received successfully
    LLM attempt 1 response malformed: 'Δοκιμαστική προτροπή για δοκιμή ενσωμάτωσης LLM...'
    LLM call attempt 1 returned empty response.
    Retrying in 3s...
    LLM call attempt 2/3...
Sending request to OpenRouter API using model: google/gemma-3-4b-it:free
    LLM call attempt 2/3...
Sending request to OpenRouter API using model: google/gemma-3-4b-it:free
API response received successfully
    LLM attempt 2 response malformed: 'Πρόκληση και καινοτομία...'
    LLM call attempt 2 returned empty response.
    Retrying in 3s...
API response received successfully
    LLM attempt 2 response malformed: 'Πρόκληση και καινοτομία...'
   

## 6. Κύρια Συνάρτηση για Εξαγωγή Τίτλων

Ορίζουμε τη συνάρτηση που συνδυάζει όλες τις προηγούμενες για την εξαγωγή ενοποιημένων τίτλων από πολλαπλά έγγραφα.

In [20]:
def get_consolidated_title_from_llm(document_summaries: list, method_name: str = "") -> str:
    """
    Sends multiple document summaries to the LLM (single call) to get a consolidated theme.
    Returns a clean theme string or an error/status string.
    """
    if not document_summaries:
        return "Δεν βρέθηκαν έγγραφα για εξαγωγή τίτλου"

    print(f"    Getting consolidated theme for {len(document_summaries)} summaries using {method_name}...")

    # Δημιουργία formatted input για το LLM
    summaries_text_parts = []
    for i, summary_text in enumerate(document_summaries):
        if not summary_text or not isinstance(summary_text, str):
            print(f"    Skipping invalid summary (index {i}).")
            continue 
        # Περιορισμός μεγέθους κάθε περίληψης για το LLM
        truncated_summary = summary_text[:500] + "..." if len(summary_text) > 500 else summary_text
        summaries_text_parts.append(f"Περίληψη {i+1}:\n{truncated_summary}")
    
    if not summaries_text_parts:
        return "Δεν υπάρχουν έγκυρες περιλήψεις για επεξεργασία"

    summaries_block = "\n\n".join(summaries_text_parts)
    num_actual_summaries = len(summaries_text_parts)
    
    # Δημιουργία του prompt
    prompt = LLM_MULTI_DOC_PROMPT_TEMPLATE.replace("{NUM_SUMMARIES}", str(num_actual_summaries))
    prompt = prompt.replace("{SUMMARIES_TEXT}", summaries_block)

    # LLM call και parsing
    llm_response_raw = call_actual_llm(prompt) 
    final_title = parse_llm_response(llm_response_raw)
    
    print(f"    Consolidated title ({method_name}): {final_title}")
    return final_title

# Δοκιμή της συνάρτησης με sample data
if df_full is not None and len(df_full) > 0:
    print("\nTesting consolidated title extraction...")
    sample_summaries = df_full[TEXT_COLUMN_TO_USE].head(3).tolist()
    sample_summaries = [s for s in sample_summaries if s and isinstance(s, str)]
    
    if sample_summaries:
        sample_title = get_consolidated_title_from_llm(sample_summaries, "TEST")
        print(f"Sample title extracted: {sample_title}")
    else:
        print("No valid sample summaries found for testing")


Testing consolidated title extraction...
    Getting consolidated theme for 3 summaries using TEST...
    LLM call attempt 1/3...
Sending request to OpenRouter API using model: google/gemma-3-4b-it:free
API response received successfully
    LLM attempt 1 response malformed: ': Θέμα: Αναίρεση και εκδίκαση ποινικών αποφάσεων...'
    LLM call attempt 1 returned empty response.
    Retrying in 3s...
API response received successfully
    LLM attempt 1 response malformed: ': Θέμα: Αναίρεση και εκδίκαση ποινικών αποφάσεων...'
    LLM call attempt 1 returned empty response.
    Retrying in 3s...
    LLM call attempt 2/3...
Sending request to OpenRouter API using model: google/gemma-3-4b-it:free
    LLM call attempt 2/3...
Sending request to OpenRouter API using model: google/gemma-3-4b-it:free
API response received successfully
    LLM attempt 2 response malformed: ':Θέμα: Αίτηση αναίρεσης και επιχειρήματα νομικού ενδιαφέροντος...'
    LLM call attempt 2 returned empty response.
    Retryin

## 7. Κύρια Διαδικασία Εξαγωγής Τίτλων

Εκτελούμε τη διαδικασία εξαγωγής τίτλων για όλα τα clusters χρησιμοποιώντας τις δύο μεθόδους επιλογής εγγράφων.

In [21]:
def process_all_clusters():
    """Main function to generate titles for clusters."""
    if df_full is None:
        print("No data loaded. Cannot process clusters.")
        return None
        
    # Προσδιορισμός clusters προς επεξεργασία
    unique_cluster_ids_from_csv = sorted(df_full['cluster_id'].unique())
    print(f"\nFound {len(unique_cluster_ids_from_csv)} unique clusters in CSV: {unique_cluster_ids_from_csv}")
    
    if not CLUSTERS_TO_PROCESS_CENTROID and not CLUSTERS_TO_PROCESS_RANDOM:
        clusters_to_iterate = unique_cluster_ids_from_csv
        print("Processing all clusters for both methods (as specific lists are empty).")
    else:
        clusters_to_iterate_set = set()
        if CLUSTERS_TO_PROCESS_CENTROID:
            clusters_to_iterate_set.update(CLUSTERS_TO_PROCESS_CENTROID)
        else:
            clusters_to_iterate_set.update(unique_cluster_ids_from_csv)
            
        if CLUSTERS_TO_PROCESS_RANDOM:
            clusters_to_iterate_set.update(CLUSTERS_TO_PROCESS_RANDOM)
        else:
            clusters_to_iterate_set.update(unique_cluster_ids_from_csv)
            
        clusters_to_iterate = sorted([cid for cid in clusters_to_iterate_set if cid in unique_cluster_ids_from_csv])

    print(f"Processing clusters: {clusters_to_iterate}")
    print("-" * 50)

    results = []

    for cluster_id in clusters_to_iterate:
        print(f"\nProcessing Cluster ID: {cluster_id}")
        
        # Φιλτράρισμα εγγράφων του cluster
        df_cluster = df_full[df_full['cluster_id'] == cluster_id].copy()
        cluster_size = len(df_cluster)
        print(f"  Cluster size: {cluster_size} documents")
        
        if cluster_size == 0:
            print(f"  Warning: Cluster {cluster_id} is empty.")
            results.append({
                "cluster_id": cluster_id,
                "title_centroid": "Άδειο cluster",
                "title_random": "Άδειο cluster",
                "cluster_size": 0
            })
            continue
        
        # Εξαγωγή TF-IDF vectors για το cluster
        original_indices_in_cluster = df_cluster.index 
        X_cluster_docs = np.array([])
        
        try:
            X_cluster_docs = X_full[original_indices_in_cluster]
            if X_cluster_docs.ndim == 1 and X_cluster_docs.shape[0] > 0: 
                X_cluster_docs = X_cluster_docs.reshape(1, -1)
        except IndexError:
            print(f"    Error: Index out of bounds for TF-IDF vectors in cluster {cluster_id}")
            X_cluster_docs = np.array([])
        
        # Έλεγχος ορίων K-means model
        if cluster_id >= k_for_refit: 
            print(f"  Skipping cluster_id {cluster_id} as it's out of bounds for K-means model (K={k_for_refit}).")
            results.append({
                "cluster_id": cluster_id,
                "title_centroid": "Σφάλμα: ID Συστάδας εκτός ορίων μοντέλου",
                "title_random": "Σφάλμα: ID Συστάδας εκτός ορίων μοντέλου",
                "cluster_size": cluster_size
            })
            continue

        current_centroid_vector = kmeans_model.cluster_centers_[cluster_id]

        # Method 1: Centroid-based selection
        title_centroid = "N/A - Δεν επεξεργάστηκε"
        run_method_1 = (not CLUSTERS_TO_PROCESS_CENTROID) or (cluster_id in CLUSTERS_TO_PROCESS_CENTROID)

        if run_method_1:
            print("  Method 1: Using 3 documents closest to centroid...")
            if X_cluster_docs.shape[0] > 0:
                closest_docs_df = get_docs_closest_to_centroid(df_cluster, X_cluster_docs, current_centroid_vector, n_docs=3)
                centroid_summaries = closest_docs_df[TEXT_COLUMN_TO_USE].tolist()
                
                if centroid_summaries:
                    if len(centroid_summaries) < 3:
                        print(f"    Warning: Only {len(centroid_summaries)} docs available for centroid method.")
                    title_centroid = get_consolidated_title_from_llm(centroid_summaries, "Centroid")
                else:
                    title_centroid = "N/A - Δεν βρέθηκαν έγγραφα για μέθοδο κεντροειδούς"
            else: 
                print(f"    Warning: No TF-IDF vectors available for cluster {cluster_id}")
                title_centroid = "N/A - Μη διαθέσιμα TF-IDF vectors"
        
        print(f"  Τίτλος (Κεντροειδής): {title_centroid}")

        # Method 2: Random selection
        title_random = "N/A - Δεν επεξεργάστηκε"
        run_method_2 = (not CLUSTERS_TO_PROCESS_RANDOM) or (cluster_id in CLUSTERS_TO_PROCESS_RANDOM)

        if run_method_2:
            print("  Method 2: Using 3 random documents from cluster...")
            random_docs_df = get_random_docs_from_cluster(df_cluster, n_docs=3, random_state=RANDOM_STATE)
            random_summaries = random_docs_df[TEXT_COLUMN_TO_USE].tolist()

            if random_summaries:
                if len(random_summaries) < 3:
                    print(f"    Warning: Only {len(random_summaries)} docs available for random method.")
                title_random = get_consolidated_title_from_llm(random_summaries, "Random")
            else:
                title_random = "N/A - Δεν βρέθηκαν έγγραφα για τυχαία μέθοδο"
        
        print(f"  Τίτλος (Τυχαία): {title_random}")
        
        # Αποθήκευση αποτελεσμάτων
        results.append({
            "cluster_id": cluster_id,
            "title_centroid": title_centroid,
            "title_random": title_random,
            "cluster_size": cluster_size
        })
        print("-" * 30)

    return results

# Εκτέλεση της διαδικασίας
if df_full is not None and 'kmeans_model' in locals():
    print("\n=== Starting cluster title extraction process ===")
    cluster_results = process_all_clusters()
    
    if cluster_results:
        print(f"\n=== Processing completed for {len(cluster_results)} clusters ===")
    else:
        print("\n=== No results generated ===")
else:
    print("Cannot proceed: Data or model not available")
    cluster_results = None


=== Starting cluster title extraction process ===

Found 21 unique clusters in CSV: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Processing all clusters for both methods (as specific lists are empty).
Processing clusters: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
--------------------------------------------------

Processing Cluster ID: 0
  Cluster size: 264 documents
  Method 1: Using 3 documents closest to centroid...
    Getting consolidated theme for 3 summaries using Centroid...
    LLM call attempt 1/3...
Sending request to OpenRouter API using model: google/gemma-3-4b-it:free
API response received successfully
    LLM attempt 1 successful and response format seems OK.
    Consolidated title (Centroid): Λόγοι αναίρεσης σε ποινικές υποθέσεις
  Τίτλος (Κεντροειδής): Λόγοι αναίρεσης σε ποινικές υποθέσεις
  Method 2: Using 3 random documents from cluster...
    Getting consolidated theme for 3 summaries using Random...
 

## 8. Αποτελέσματα και Αποθήκευση

Εμφανίζουμε τα αποτελέσματα της εξαγωγής τίτλων και τα αποθηκεύουμε σε CSV αρχείο.

In [22]:
def display_and_save_results(results):
    """Display results and save to CSV."""
    if not results:
        print("No results to display or save.")
        return False
        
    print("\n\n--- Περίληψη Δημιουργημένων Τίτλων ---")
    results_df = pd.DataFrame(results)
    
    # Εμφάνιση αποτελεσμάτων
    pd.set_option('display.max_colwidth', 80)
    pd.set_option('display.width', None)
    print(results_df.to_string(index=False))
    
    # Στατιστικά αποτελεσμάτων
    print(f"\n--- Στατιστικά Αποτελεσμάτων ---")
    print(f"Συνολικά clusters επεξεργασμένα: {len(results_df)}")
    
    # Επιτυχημένες εξαγωγές τίτλων (χωρίς error messages)
    successful_centroid = results_df[~results_df['title_centroid'].str.contains('N/A|Σφάλμα|Αποτυχία', na=False)].shape[0]
    successful_random = results_df[~results_df['title_random'].str.contains('N/A|Σφάλμα|Αποτυχία', na=False)].shape[0]
    
    print(f"Επιτυχημένες εξαγωγές (Κεντροειδής): {successful_centroid}/{len(results_df)} ({successful_centroid/len(results_df)*100:.1f}%)")
    print(f"Επιτυχημένες εξαγωγές (Τυχαία): {successful_random}/{len(results_df)} ({successful_random/len(results_df)*100:.1f}%)")
    
    # Στατιστικά cluster sizes
    print(f"Συνολικά documents: {results_df['cluster_size'].sum()}")
    print(f"Μέσο μέγεθος cluster: {results_df['cluster_size'].mean():.1f}")
    print(f"Μέγιστο μέγεθος cluster: {results_df['cluster_size'].max()}")
    print(f"Ελάχιστο μέγεθος cluster: {results_df['cluster_size'].min()}")
    
    # Αποθήκευση σε CSV
    output_csv_filename = "cluster_titles_summary.csv"
    output_csv_path = os.path.join('.', output_csv_filename)
    
    try:
        results_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
        print(f"\nΤα αποτελέσματα αποθηκεύτηκαν στο: {os.path.abspath(output_csv_path)}")
        return True
    except Exception as e:
        print(f"\nΣφάλμα κατά την αποθήκευση των αποτελεσμάτων σε CSV: {e}")
        return False

# Εμφάνιση και αποθήκευση αποτελεσμάτων
if cluster_results:
    success = display_and_save_results(cluster_results)
    if success:
        print(f"\n=== Διαδικασία ολοκληρώθηκε επιτυχώς ===")
    else:
        print(f"\n=== Διαδικασία ολοκληρώθηκε με σφάλματα αποθήκευσης ===")
else:
    print("No results to process.")



--- Περίληψη Δημιουργημένων Τίτλων ---
 cluster_id                                                           title_centroid                                                      title_random  cluster_size
          0                                    Λόγοι αναίρεσης σε ποινικές υποθέσεις                        Αιτήσεις αναίρεσης και έλλειψη αιτιολογίας           264
          1 Καταπορριφθέντες λόγοι αναιρέσεως καθυστέρησης καταβολής χρέους δημοσίου Αναίρεση καταδικαστικών αποφάσεων για καθυστέρηση καταβολής χρεών            47
          2                                 Απορρίψεις εφέσεων για εκπρόθεσμη ασκήση                       Ακυρότητα και αιτιολόγηση αποφάσεων εφέσεων           342
          3                          Αναιρέσεις και έλλειψη αιτιολογίας σε αποφάσεις                          Ακυρότητα και αναίρεση νομικών αποφάσεων           445
          4                                             Απαράδεκτη συζήτηση αιτήσεων    Απαίτηση νομιμότητας και δεσμευτικότητας δικασ

## 9. Ανάλυση και Σύγκριση Τίτλων

Αναλύουμε τα αποτελέσματα και συγκρίνουμε τους τίτλους που προέκυψαν από τις δύο μεθόδους.

In [23]:
def analyze_title_differences(results_df):
    """Analyze differences between centroid and random methods."""
    if results_df is None or results_df.empty:
        print("No data to analyze.")
        return
        
    print("\n--- Ανάλυση Διαφορών μεταξύ Μεθόδων ---")
    
    # Φιλτράρισμα επιτυχημένων extractions
    valid_data = results_df[
        (~results_df['title_centroid'].str.contains('N/A|Σφάλμα|Αποτυχία', na=False)) &
        (~results_df['title_random'].str.contains('N/A|Σφάλμα|Αποτυχία', na=False))
    ].copy()
    
    if valid_data.empty:
        print("Δεν υπάρχουν έγκυρα δεδομένα για σύγκριση.")
        return
        
    print(f"Clusters με έγκυρους τίτλους από αμφότερες τις μεθόδους: {len(valid_data)}")
    
    # Υπολογισμός ομοιότητας τίτλων (απλή σύγκριση)
    identical_titles = 0
    similar_keywords = 0
    completely_different = 0
    
    print(f"\n--- Παραδείγματα Συγκρίσεων ---")
    
    for idx, row in valid_data.head(10).iterrows():  # Εμφάνιση των πρώτων 10
        centroid_title = row['title_centroid']
        random_title = row['title_random']
        cluster_id = row['cluster_id']
        
        print(f"\nCluster {cluster_id}:")
        print(f"  Κεντροειδής: {centroid_title}")
        print(f"  Τυχαία:     {random_title}")
        
        # Απλή ανάλυση ομοιότητας
        if centroid_title.lower() == random_title.lower():
            identical_titles += 1
            print(f"  Αξιολόγηση: Ταυτόσημοι τίτλοι")
        else:
            # Έλεγχος κοινών λέξεων
            centroid_words = set(centroid_title.lower().split())
            random_words = set(random_title.lower().split())
            common_words = centroid_words.intersection(random_words)
            
            if len(common_words) >= 2:  # Τουλάχιστον 2 κοινές λέξεις
                similar_keywords += 1
                print(f"  Αξιολόγηση: Παρόμοιοι (κοινές λέξεις: {', '.join(common_words)})")
            else:
                completely_different += 1
                print(f"  Αξιολόγηση: Διαφορετικοί")
    
    # Στατιστικά ομοιότητας
    total_valid = len(valid_data)
    print(f"\n--- Στατιστικά Ομοιότητας (για όλα τα έγκυρα clusters) ---")
    
    # Υπολογισμός για όλα τα έγκυρα clusters
    all_identical = 0
    all_similar = 0
    all_different = 0
    
    for idx, row in valid_data.iterrows():
        centroid_title = row['title_centroid']
        random_title = row['title_random']
        
        if centroid_title.lower() == random_title.lower():
            all_identical += 1
        else:
            centroid_words = set(centroid_title.lower().split())
            random_words = set(random_title.lower().split())
            common_words = centroid_words.intersection(random_words)
            
            if len(common_words) >= 2:
                all_similar += 1
            else:
                all_different += 1
    
    print(f"Ταυτόσημοι τίτλοι: {all_identical}/{total_valid} ({all_identical/total_valid*100:.1f}%)")
    print(f"Παρόμοιοι τίτλοι: {all_similar}/{total_valid} ({all_similar/total_valid*100:.1f}%)")
    print(f"Διαφορετικοί τίτλοι: {all_different}/{total_valid} ({all_different/total_valid*100:.1f}%)")

# Ανάλυση των αποτελεσμάτων
if cluster_results:
    results_df_for_analysis = pd.DataFrame(cluster_results)
    analyze_title_differences(results_df_for_analysis)


--- Ανάλυση Διαφορών μεταξύ Μεθόδων ---
Clusters με έγκυρους τίτλους από αμφότερες τις μεθόδους: 20

--- Παραδείγματα Συγκρίσεων ---

Cluster 0:
  Κεντροειδής: Λόγοι αναίρεσης σε ποινικές υποθέσεις
  Τυχαία:     Αιτήσεις αναίρεσης και έλλειψη αιτιολογίας
  Αξιολόγηση: Διαφορετικοί

Cluster 1:
  Κεντροειδής: Καταπορριφθέντες λόγοι αναιρέσεως καθυστέρησης καταβολής χρέους δημοσίου
  Τυχαία:     Αναίρεση καταδικαστικών αποφάσεων για καθυστέρηση καταβολής χρεών
  Αξιολόγηση: Διαφορετικοί

Cluster 2:
  Κεντροειδής: Απορρίψεις εφέσεων για εκπρόθεσμη ασκήση
  Τυχαία:     Ακυρότητα και αιτιολόγηση αποφάσεων εφέσεων
  Αξιολόγηση: Διαφορετικοί

Cluster 3:
  Κεντροειδής: Αναιρέσεις και έλλειψη αιτιολογίας σε αποφάσεις
  Τυχαία:     Ακυρότητα και αναίρεση νομικών αποφάσεων
  Αξιολόγηση: Διαφορετικοί

Cluster 4:
  Κεντροειδής: Απαράδεκτη συζήτηση αιτήσεων
  Τυχαία:     Απαίτηση νομιμότητας και δεσμευτικότητας δικαστικών συζητήσεων
  Αξιολόγηση: Διαφορετικοί

Cluster 5:
  Κεντροειδής: Απολύσεις και

## 10. Συμπεράσματα και Παρατηρήσεις

### Αποτελέσματα της Εξαγωγής Τίτλων με LLM

Βασισμένα στην ανάλυση που πραγματοποιήσαμε:

**Τεχνικά Χαρακτηριστικά:**
- **LLM Model**: Google Gemma-3-4b-it:free μέσω OpenRouter API
- **Input Data**: Clusters από K-means clustering (K=21)
- **Επιλογή Εγγράφων**: Δύο μέθοδοι - κεντροειδής και τυχαία επιλογή
- **Prompt Engineering**: Δομημένο prompt για εξαγωγή σύντομων περιγραφικών τίτλων

**Αξιολόγηση των Μεθόδων:**

1. **Μέθοδος Κεντροειδούς:**
   - Επιλέγει έγγραφα πιο αντιπροσωπευτικά του cluster
   - Τείνει να παράγει πιο συγκεκριμένους και τεχνικούς τίτλους
   - Βασίζεται στη μαθηματική αναπαράσταση του cluster

2. **Τυχαία Μέθοδος:**
   - Παρέχει ποικιλομορφία στην επιλογή εγγράφων
   - Μπορεί να αποκαλύψει διαφορετικές πτυχές του cluster
   - Λιγότερο προβλέψιμη αλλά δυνητικά πιο ανθρώπινη προσέγγιση

**Παρατηρήσεις:**
- Η ποιότητα των τίτλων εξαρτάται από την ποιότητα των εισαγωγικών κειμένων
- Τα μεγαλύτερα clusters τείνουν να παράγουν πιο σταθερούς τίτλους
- Η σύγκριση μεταξύ των δύο μεθόδων αποκαλύπτει τη συνοχή του cluster

**Εφαρμογές:**
- Αυτοματοποιημένη κατηγοριοποίηση νομικών εγγράφων
- Δημιουργία taxonomies για νομικά θέματα
- Υποστήριξη στην αναζήτηση και οργάνωση νομικού περιεχομένου
- Βάση για περαιτέρω ανάλυση θεμάτων στη νομολογία

Η συγκεκριμένη προσέγγιση αποτελεί ένα σημαντικό βήμα στην αυτοματοποίηση της κατεγοριοποίησης νομικών κειμένων, παρέχοντας ένα εργαλείο για την καλύτερη κατανόηση και οργάνωση μεγάλων συλλογών νομικών εγγράφων.